posibilitati de imbunatatire a recunoasterii textului

In [ ]:
import cv2
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials
from array import array
import os
from PIL import Image
import sys
import time
import os
from io import BytesIO


RED = "\033[31m"
GREEN = "\033[32m"
YELLOW = "\033[33m"
BLUE = "\033[34m"
MAGENTA = "\033[35m"
CYAN = "\033[36m"
WHITE = "\033[37m"
RESET = "\033[0m"

os.environ["VISION_KEY"] ="Your_key"
os.environ["VISION_ENDPOINT"] ="https://berindeiepaulmihai.cognitiveservices.azure.com/"
subscription_key = os.environ["VISION_KEY"]
endpoint = os.environ["VISION_ENDPOINT"]
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [2]:
def levenshteinDistance(str1, str2):
    # programare dinamica :(

    n = len(str1) # lungimea sirului sursa
    m = len(str2) # lungimea sirului target
    dp = [[0 for _ in range(m + 1)] for _ in range(n + 1)]

    # se initializeaza prima linie din matrice cu numere de la 0 la n
    for i in range(n + 1):
        dp[i][0] = i

    # se initializeaza prima coloana din matrice cu numere de la 0 la m
    for j in range(m + 1):
        dp[0][j] = j

    for i in range(1, n + 1):
        for j in range(1, m + 1):
            if str1[i - 1] == str2[j - 1]:  # daca valorile sunt egale atunci
                dp[i][j] = dp[i - 1][j - 1] # atunci se ia valoare din stanga sus
            else:
                dp[i][j] = 1 + min(dp[i][j - 1], dp[i - 1][j], dp[i - 1][j - 1]) # se ia valoare din celilele de deasupra si stanga
    return dp[n][m]

In [3]:
import numpy as np

def enhance_contrast(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    img_enhanced = clahe.apply(img)
    return img_enhanced

def remove_noise(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    kernel = np.ones((1, 1), np.uint8)
    img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
    img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
    return img

def binarize_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    _, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    return binary


files = os.listdir('img')
folder = 'img'


groundTruth = {
                'test1.jpg': ['Acesta este un test', 'Stiu ca nu scriu foarte frumos!'],
                'test2.jpg': ['LIGHT', 'BOX'],
                'test3.png': ['CEVA', 'IN', 'AI'],
               }
OCR_rez = dict()

for file in files:

    image_path = os.path.join(folder, file)
    img_enhanced = enhance_contrast(image_path)
    is_success, buffer = cv2.imencode(".jpg", img_enhanced)
    img_bytes = BytesIO(buffer)


    read_response = computervision_client.read_in_stream(
    image=img_bytes,
    mode="Printed",
    raw=True
    )

    operation_id = read_response.headers['Operation-Location'].split('/')[-1]
    while True:
        read_result = computervision_client.get_read_result(operation_id)
        if read_result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)


    result = []
    if read_result.status == OperationStatusCodes.succeeded:
        for text_result in read_result.analyze_result.read_results:
            for line in text_result.lines:
                result.append(line.text)

    if file not in OCR_rez:
        OCR_rez[file] = result

print(groundTruth)
print(OCR_rez)


{'test1.jpg': ['Acesta este un test', 'Stiu ca nu scriu foarte frumos!'], 'test2.jpg': ['LIGHT', 'BOX'], 'test3.png': ['CEVA', 'IN', 'AI']}
{'test1.jpg': ['Acesta este un test', 'Itin ca nu solíu facete frumos!'], 'test2.jpg': ['LIGHT', 'BOX'], 'test3.png': ['CEVA', '6', 'Al']}


In [4]:
print("Calitatea procesului la nivel de caracter:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        wordsGroundTruth = line1.split(' ')
        wordsOCR = line2.split(' ')
        for w1, w2 in zip(wordsGroundTruth, wordsOCR):
            print(f"        {GREEN}{w1}{RESET} VS {RED}{w2}{RESET} - diferenta: {levenshteinDistance(w1, w2)}")

print("Calitatea procesului la nivel de cuvant:")
for file in OCR_rez:
    print(f"    {CYAN}{file}{RESET}")
    noOfCorrectWords = 0
    for line1, line2 in zip(groundTruth[file], OCR_rez[file]):
        print(f"        {GREEN}{line1}{RESET}  {RED}{line2}{RESET}")
        for w1, w2 in zip(line1.split(' '), line2.split(' ')):
            if w1 == w2:
                noOfCorrectWords += 1
    print(f"        Numarul de cuvinte corecte este {noOfCorrectWords}")


Calitatea procesului la nivel de caracter:
    test1.jpg
        Acesta VS Acesta - diferenta: 0
        este VS este - diferenta: 0
        un VS un - diferenta: 0
        test VS test - diferenta: 0
        Stiu VS Itin - diferenta: 2
        ca VS ca - diferenta: 0
        nu VS nu - diferenta: 0
        scriu VS solíu - diferenta: 3
        foarte VS facete - diferenta: 3
        frumos! VS frumos! - diferenta: 0
    test2.jpg
        LIGHT VS LIGHT - diferenta: 0
        BOX VS BOX - diferenta: 0
    test3.png
        CEVA VS CEVA - diferenta: 0
        IN VS 6 - diferenta: 2
        AI VS Al - diferenta: 1
Calitatea procesului la nivel de cuvant:
    test1.jpg
        Acesta este un test  Acesta este un test
        Stiu ca nu scriu foarte frumos!  Itin ca nu solíu facete frumos!
        Numarul de cuvinte corecte este 7
    test2.jpg
        LIGHT  LIGHT
        BOX  BOX
        Numarul de cuvinte corecte este 2
    test3.png
        CEVA  CEVA
        IN  6
        AI  Al
     

Pentru inbunatatire se poate normaliza imaginea, acest proces ar trebui sa ofere o performanta cu 20-30% mai buna. Eu am folosit o trecere in GreyScale